Mount Google Drive

In [89]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
train_dir = '/content/drive/MyDrive/Skin Disease_dataset/TRAIN'
test_dir = '/content/drive/MyDrive/Skin Disease_dataset/TEST'

Augmentasi Data

In [91]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

In [92]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

In [93]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [94]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training')

Found 640 images belonging to 10 classes.


In [95]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation')

Found 160 images belonging to 10 classes.


In [96]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical')

Found 200 images belonging to 10 classes.


Model MobileNetV2

In [97]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [98]:
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(10, activation='softmax')(x)

In [99]:
model = Model(inputs=base_model.input, outputs=predictions)

In [100]:
opt = Adam(learning_rate=1e-4)  # Set initial learning rate
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [101]:
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

In [102]:
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[LearningRateScheduler(lr_scheduler, verbose=1)]
)


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/50
20/20 [==============================] - 20s 802ms/step - loss: 2.5366 - accuracy: 0.1484 - val_loss: 2.0301 - val_accuracy: 0.2313 - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/50
20/20 [==============================] - 15s 744ms/step - loss: 1.9035 - accuracy: 0.3219 - val_loss: 1.7870 - val_accuracy: 0.3063 - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/50
20/20 [==============================] - 15s 746ms/step - loss: 1.6363 - accuracy: 0.4281 - val_loss: 1.7805 - val_accuracy: 0.2812 - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/50
20/20 [==============================] - 15s 757ms/step - loss: 1.4666 - accuracy: 0.4531 - val_loss: 1.7270 - val_accuracy: 0.3438 - lr: 1.0000e-04

Epoch 5: LearningRateScheduler setting 

In [103]:
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test Accuracy: {test_acc * 100:.2f}%')

7/7 [==============================] - 2s 300ms/step - loss: 1.6425 - accuracy: 0.3650
Test Accuracy: 36.50%
